In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from utility.types import RebalanceFrequencyEnum, AllocationMethodsEnum, RegimeDetectionModels

from data.universe import Universe
from data.benchmark import Benchmark

from backtest.backtest import Backtester


In [2]:
bench_obj = Benchmark(RebalanceFrequencyEnum.MONTH_END)
bench_obj.benchmark_perf

2016-05-19    1.000000
2016-05-20    1.000000
2016-05-23    1.000000
2016-05-24    1.000000
2016-05-25    1.000000
                ...   
2024-03-01    1.658112
2024-03-04    1.661620
2024-03-05    1.655421
2024-03-06    1.660339
2024-03-07    1.671945
Freq: B, Name: benchmark_perf, Length: 2036, dtype: float64

In [ ]:
univ_obj = Universe()
universe = univ_obj.get_universe_returns()
universe

In [ ]:
d =univ_obj.get_universe_returns()
d

In [ ]:

bk_tester = Backtester(
    universe_returns=univ_obj.get_universe_returns(),
    market_returns=universe["ETF_SPX"],
    benchmark_returns=bench_obj.benchmark_returns,
)


ptf_and_bench, ptf_weights_equal_weight, regimes, metrics_df = bk_tester.run_backtest(
    allocation_type=AllocationMethodsEnum.EQUALLY_WEIGHTED,
    rebalance_frequency=RebalanceFrequencyEnum.MONTH_END,
    market_regime_model=RegimeDetectionModels.HIDDEN_MARKOV_MODEL,
    transaction_cost_by_securities={k: 0.0015 for k in universe.columns},
    bearish_leverage_by_securities={'MONTAIRE': 1,
                                    'ETF_GROWTH': 1,
                                    'ETF_VALUE': 1,
                                    'ETF_CAC': 0.6,
                                    'ETF_SPX': 0.6},
    bullish_leverage_by_securities={'MONTAIRE': 1,
                                    'ETF_GROWTH': 1,
                                    'ETF_VALUE': 1,
                                    'ETF_CAC': 1,
                                    'ETF_SPX': 1},
    verbose=False,
)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 10))


ax.plot(regimes.index,regimes.next_beta, label="Beta")
ax_l = ax.twinx()
ax_l.fill_between(
    regimes.index,
    regimes["Regime"],
    alpha=0.2,
    color="red",
    step="pre",
)
ax.set_xlabel("Datetime", fontsize=15)
ax.set_ylabel("Beta", fontsize=15)
ax.set_title(f"Dynamic beta evolution", fontsize=20)
ax.grid()
ax.legend(fontsize=15)

